In [ ]:
import tweepy
import pandas as pd
import oandapyV20
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.instruments import InstrumentsCandles


In [ ]:
# define Twitter API credentials and set up the Twitter API client using tweepy
consumer_key = "YOUR_CONSUMER_KEY"
consumer_secret = "YOUR_CONSUMER_SECRET"
access_token = "YOUR_ACCESS_TOKEN"
access_token_secret = "YOUR_ACCESS_TOKEN_SECRET"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create an instance of the Twitter API client
api_twitter = tweepy.API(auth)


In [ ]:
# Define a function to fetch tweets for a given query
def fetch_tweets(query, count=100):
    tweets = api_twitter.search(query, count=count)
    df = pd.DataFrame(columns=["text", "created_at"])
    for tweet in tweets:
        df = df.append({"text": tweet.text, "created_at": tweet.created_at}, ignore_index=True)
    return df

# Define a query for fetching tweets
query = "forex trading"

# Fetch tweets
tweets_df = fetch_tweets(query, count=100)


In [ ]:
# Define OANDA API credentials
api_token = "YOUR_API_TOKEN"
account_id = "YOUR_ACCOUNT_ID"

# Create an instance of the OANDA API client
api = API(access_token=api_token)

In [ ]:
# Define the forex major pairs you want to collect data for
currency_pairs = ["EUR_USD", "GBP_USD", "USD_JPY"]

In [ ]:
# Define the timeframe for the data (e.g., H1 for 1-hour candles)
timeframe = "H1"

# Define the number of candles to retrieve
num_candles = 500

In [ ]:
# Define a function to fetch candles data for a given currency pair
def fetch_candles(currency_pair):
    params = {
        "count": num_candles,
        "granularity": timeframe,
    }
    r = InstrumentsCandles(instrument=currency_pair, params=params)
    try:
        response = api.request(r)
        candles = response["candles"]
        df = pd.DataFrame(candles)
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        return df
    except V20Error as e:
        print(f"Error fetching data for {currency_pair}: {e}")



In [ ]:
# Fetch candles data for each currency pair
data = {}
for currency_pair in currency_pairs:
    df = fetch_candles(currency_pair)
    data[currency_pair] = df

# Print the data for the first currency pair (EUR/USD) as an example
print(data["EUR_USD"])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# having collected the forex and Twitter data in separate DataFrames
forex_data = ...
twitter_data = ...

# Clean the data
forex_data_cleaned = forex_data.dropna()      # remove any rows with missing values
twitter_data_cleaned = twitter_data.dropna()  # remove any rows with missing values

# Merge the forex and Twitter data based on a common column, such as timestamp
merged_data = pd.merge(forex_data_cleaned, twitter_data_cleaned, on="timestamp")

# Prepare features and target variables
features = merged_data[["forex_feature1", "forex_feature2", "twitter_feature1", "twitter_feature2"]]
target = merged_data["target_variable"]

# Split the data into training and testing sets
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size=0.2, random_state=42
)

# Normalize or scale the features
scaler = StandardScaler()   # normalize or scale the features
train_features_scaled = scaler.fit_transform(train_features)
test_features_scaled = scaler.transform(test_features)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.metrics import accuracy_score

In [ ]:
# preprocessed data and target variables from the previous step
train_features_scaled = ...
train_target = ...
test_features_scaled = ...
test_target = ...

# Feature Engineering
# You can add technical indicators, macroeconomic indicators, sentiment indicators, etc., based on your domain expertise.
# Here's an example of adding a moving average feature:
window_size = 5
moving_average = train_features_scaled[:, 0].rolling(window=window_size).mean().values

# Concatenate the moving average feature with the existing features
train_features_engineered = np.column_stack((train_features_scaled, moving_average))

# Model Selection and Training
model = Sequential()
model.add(LSTM(64, input_shape=(train_features_engineered.shape[1],)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_features_engineered, train_target, epochs=10, batch_size=32)

# Evaluation
test_features_engineered = np.column_stack(
    (test_features_scaled, test_features_scaled[:, 0].rolling(window=window_size).mean().values)
)
predictions = model.predict(test_features_engineered)
binary_predictions = np.where(predictions > 0.5, 1, 0)
accuracy = accuracy_score(test_target, binary_predictions)
print("Accuracy:", accuracy)


In [ ]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# Assuming you have the labeled sentiment data for training and testing
sentiment_data = pd.read_csv('sentiment_data.csv')

# Split the data into training and testing sets
train_data, test_data = train_test_split(sentiment_data, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['text'])

# Convert text data to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

# Pad sequences to a fixed length
max_sequence_length = 100
train_data_padded = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_data_padded = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Convert sentiment labels to binary representation (0 or 1)
train_labels = np.array(train_data['label'])
test_labels = np.array(test_data['label'])

# Model Training
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_sequence_length))
model.add(LSTM(64, return_sequences=True))
model.add(GlobalMaxPooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_data_padded, train_labels, validation_data=(test_data_padded, test_labels), epochs=10, batch_size=32)

# Prediction
text_samples = ["I love this product!", "This movie is terrible."]
sequences = tokenizer.texts_to_sequences(text_samples)
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
predictions = model.predict(padded_sequences)

for text, prediction in zip(text_samples, predictions):
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    print(f"Text: {text}\nSentiment: {sentiment}\n")

In [ ]:
# Assuming you have obtained the predictions from sentiment analysis model and momentum analysis signals

def generate_signals(sentiment_predictions, momentum_signals, threshold_sentiment=0.5, threshold_momentum=0.7):
    """
    Generate buy or sell signals based on sentiment predictions and momentum signals.
    
    Args:
        sentiment_predictions (np.array): Predictions from sentiment analysis model.
        momentum_signals (np.array): Momentum analysis signals.
        threshold_sentiment (float): Threshold for sentiment predictions to trigger a signal.
        threshold_momentum (float): Threshold for momentum signals to trigger a signal.
    
    Returns:
        np.array: Buy (1), sell (-1), or hold (0) signals.
    """
    signals = np.zeros_like(sentiment_predictions)
    
    for i in range(len(signals)):
        if sentiment_predictions[i] > threshold_sentiment and momentum_signals[i] > threshold_momentum:
            signals[i] = 1  # Buy signal
        elif sentiment_predictions[i] < 1 - threshold_sentiment and momentum_signals[i] < 1 - threshold_momentum:
            signals[i] = -1  # Sell signal
    
    return signals

# Assuming you have sentiment predictions and momentum signals as numpy arrays
sentiment_predictions = np.array([0.7, 0.3, 0.8, 0.6, 0.9])
momentum_signals = np.array([0.8, 0.9, 0.5, 0.6, 0.3])

# Generate signals
signals = generate_signals(sentiment_predictions, momentum_signals)

# Print the generated signals
for i, signal in enumerate(signals):
    if signal == 1:
        print(f"Signal {i + 1}: Buy")
    elif signal == -1:
        print(f"Signal {i + 1}: Sell")
    else:
        print(f"Signal {i + 1}: Hold")


In [ ]:
# Given historical forex data and signals generated from the algorithm

def backtest(historical_data, signals):
    """
    Perform backtesting on historical data using generated signals.
    
    Args:
        historical_data (pd.DataFrame): Historical forex data including price and other relevant features.
        signals (np.array): Buy (1), sell (-1), or hold (0) signals.
    
    Returns:
        pd.DataFrame: DataFrame with backtesting results, including portfolio value and returns.
    """
    # Combine signals with historical data
    signals_df = pd.DataFrame({'signal': signals}, index=historical_data.index)
    combined_data = pd.concat([historical_data, signals_df], axis=1).dropna()
    
    # Calculate daily returns based on signals
    combined_data['return'] = combined_data['signal'] * combined_data['price'].pct_change()
    
    # Calculate cumulative returns
    combined_data['cumulative_return'] = (1 + combined_data['return']).cumprod()
    
    # Calculate portfolio value
    combined_data['portfolio_value'] = combined_data['cumulative_return'] * initial_investment
    
    # Calculate benchmark returns (assuming buy and hold strategy)
    combined_data['benchmark_return'] = combined_data['price'].pct_change().cumsum() + 1
    
    # Calculate portfolio returns
    combined_data['portfolio_return'] = combined_data['portfolio_value'].pct_change()
    
    # Calculate other performance metrics (e.g., Sharpe ratio, maximum drawdown)
    # ...
    
    return combined_data

# Assuming you have historical forex data as a pandas DataFrame
historical_data = pd.DataFrame({
    'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05'],
    'price': [100, 105, 103, 108, 110]
})
historical_data['date'] = pd.to_datetime(historical_data['date'])
historical_data.set_index('date', inplace=True)

# Assuming you have generated signals as a numpy array
signals = np.array([1, -1, 0, 1, -1])

# Set initial investment amount
initial_investment = 10000

# Perform backtesting
backtest_results = backtest(historical_data, signals)

# Print the backtesting results
print(backtest_results)

# Plot the portfolio value and benchmark returns
import matplotlib.pyplot as plt

plt.plot(backtest_results['portfolio_value'], label='Portfolio Value')
plt.plot(backtest_results['benchmark_return'], label='Benchmark Returns')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.show()


In [ ]:
import tweepy
import pandas as pd
import oandapyV20
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.instruments import InstrumentsCandles
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
# Define Twitter API credentials
consumer_key = "YOUR_CONSUMER_KEY"
consumer_secret = "YOUR_CONSUMER_SECRET"
access_token = "YOUR_ACCESS_TOKEN"
access_token_secret = "YOUR_ACCESS_TOKEN_SECRET"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create an instance of the Twitter API client
api_twitter = tweepy.API(auth)

# Define OANDA API credentials
api_token = "YOUR_API_TOKEN"
account_id = "YOUR_ACCOUNT_ID"

# Create an instance of the OANDA API client
api = API(access_token=api_token)


In [ ]:
# Define a function to fetch tweets for a given query
def fetch_tweets(query, count=100):
    tweets = api_twitter.search(query, count=count)
    df = pd.DataFrame(columns=["text", "created_at"])
    for tweet in tweets:
        df = df.append({"text": tweet.text, "created_at": tweet.created_at}, ignore_index=True)
    return df

# Define a function to fetch candles data for a given currency pair
def fetch_candles(currency_pair):
    params = {
        "count": num_candles,
        "granularity": timeframe,
    }
    r = InstrumentsCandles(instrument=currency_pair, params=params)
    try:
        response = api.request(r)
        candles = response["candles"]
        df = pd.DataFrame(candles)
        df["time"] = pd.to_datetime(df["time"])
        df.set_index("time", inplace=True)
        return df
    except V20Error as e:
        print(f"Error fetching data for {currency_pair}: {e}")


In [ ]:
# Define the forex major pairs you want to collect data for
currency_pairs = ["EUR_USD", "GBP_USD", "USD_JPY"]

# Define the timeframe for the data (e.g., H1 for 1-hour candles)
timeframe = "H1"

# Define the number of candles to retrieve
num_candles = 500

# Set initial investment amount
initial_investment = 10000


In [ ]:
# Fetch tweets
query = "forex trading"
tweets_df = fetch_tweets(query, count=100)

# Fetch candles data for each currency pair
data = {}
for currency_pair in currency_pairs:
    df = fetch_candles(currency_pair)
    data[currency_pair] = df


In [ ]:
risk_per_trade = 0.02  # Percentage of account balance to risk per trade
stop_loss_factor = 0.02  # Percentage of trade amount as stop-loss
portfolio_diversification = 0.2  # Percentage of account balance to allocate for portfolio diversification


In [ ]:
def calculate_position_size(balance, risk_per_trade, entry_price, stop_loss_price):
    risk_amount = balance * risk_per_trade
    trade_amount = risk_amount / (entry_price - stop_loss_price)
    return trade_amount


In [ ]:
def calculate_stop_loss(entry_price, stop_loss_factor):
    stop_loss_price = entry_price - (entry_price * stop_loss_factor)
    return stop_loss_price


In [ ]:
def allocate_portfolio_balance(balance, portfolio_diversification):
    allocated_balance = balance * portfolio_diversification
    return allocated_balance


In [ ]:
# Assuming you have calculated the entry price and trade direction
entry_price = ...  # Replace with your calculated entry price
trade_direction = ...  # Replace with your trade direction ('buy' or 'sell')

# Calculate stop-loss price
stop_loss_price = calculate_stop_loss(entry_price, stop_loss_factor)

# Calculate position size based on risk-reward ratio
trade_amount = calculate_position_size(balance, risk_per_trade, entry_price, stop_loss_price)

# Allocate portfolio balance for diversification
allocated_balance = allocate_portfolio_balance(balance, portfolio_diversification)

# Determine the actual trade amount based on available allocated balance
if trade_amount > allocated_balance:
    trade_amount = allocated_balance

# Place the trade using the OANDA API
if trade_direction == 'buy':
    units = trade_amount
else:
    units = -trade_amount

try:
    response = api.create_order(
        account_id,
        instrument=currency_pair,
        units=units,
        type='MARKET',
        timeInForce='FOK',
    )
    print("Trade executed successfully.")
except V20Error as e:
    print(f"Error executing trade: {e}")
